<a href="https://colab.research.google.com/github/JoshuaLee0816/diffuserlite.github.io/blob/main/01_DiffuserLite_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
from google.colab import userdata

# === 1. 設定你的 GitHub 資訊 ===
GITHUB_TOKEN = userdata.get('GITHUB_TOKEN')# 建議之後更換
GITHUB_USER = "JoshuaLee0816"
GITHUB_REPO = "diffuserlite.github.io"

import os
os.chdir('/content')

# === 2. Clone 你的倉庫作為「唯一」工作目錄 ===
!rm -rf /content/{GITHUB_REPO}
!git clone https://{GITHUB_TOKEN}@github.com/{GITHUB_USER}/{GITHUB_REPO}.git

# 進入倉庫
%cd /content/{GITHUB_REPO}

# === 3. 安裝環境（在你的 Repo 目錄下安裝） ===
!pip install -e . -q  # 以可編輯模式安裝目前的倉庫
!pip install git+https://github.com/Farama-Foundation/D4RL.git --ignore-requires-python -q
!pip install "numpy>=1.26.0,<2.0.0" -q
!pip install --upgrade huggingface_hub -q  # Hugging Face 上傳用

# MuJoCo 210 安裝
!mkdir -p /root/.mujoco
!wget -q https://mujoco.org/download/mujoco210-linux-x86_64.tar.gz -O /tmp/mujoco210.tar.gz
!tar -xzf /tmp/mujoco210.tar.gz -C /root/.mujoco/
!apt-get install -qq -y libosmesa6-dev libgl1-mesa-glx libglfw3 patchelf > /dev/null 2>&1

# 設定 Git 身份
!git config --global user.email "310274movie@gmail.com"
!git config --global user.name "JoshuaLee"

print("✅ 環境已在你的 Repo 中安裝完成！現在所有變動都會記錄在此。")


Cloning into 'diffuserlite.github.io'...
remote: Enumerating objects: 401, done.
remote: Counting objects: 100% (401/401), done.
remote: Compressing objects: 100% (329/329), done.
remote: Total 401 (delta 70), reused 390 (delta 60), pack-reused 0 (from 0)
Receiving objects: 100% (401/401), 14.00 MiB | 17.83 MiB/s, done.
Resolving deltas: 100% (70/70), done.
/content/diffuserlite.github.io
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 MB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.4/56.4 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 56.3 MB/s eta 0:00:00
   ━━

In [2]:
# === Hugging Face 登入（只需執行一次）===
from huggingface_hub import login

login(token=userdata.get('HF_TOKEN'))

In [3]:
%%writefile /content/run.py
import sys, os, types, json
import mujoco
import numpy as np

shim = types.ModuleType('mujoco_py')

class ShimModel:
    def __init__(self, m):
        self._m = m
    @property
    def actuator_ctrlrange(self):
        return self._m.actuator_ctrlrange.copy()
    @property
    def nq(self): return self._m.nq
    @property
    def nv(self): return self._m.nv
    def __getattr__(self, name):
        return getattr(self._m, name)

class ShimData:
    def __init__(self, d):
        self._d = d
    @property
    def qpos(self): return self._d.qpos
    @qpos.setter
    def qpos(self, v): self._d.qpos[:] = v
    @property
    def qvel(self): return self._d.qvel
    @qvel.setter
    def qvel(self, v): self._d.qvel[:] = v
    @property
    def ctrl(self): return self._d.ctrl
    @ctrl.setter
    def ctrl(self, v): self._d.ctrl[:] = v
    def __getattr__(self, name):
        return getattr(self._d, name)

class ShimSim:
    def __init__(self, model):
        self._m = model._m
        self._d = mujoco.MjData(self._m)
        self.model = model
        self.data = ShimData(self._d)
    def step(self):
        mujoco.mj_step(self._m, self._d)
    def forward(self):
        mujoco.mj_forward(self._m, self._d)
    def get_state(self):
        return type('S',(),{'time':self._d.time,'qpos':self._d.qpos.copy(),'qvel':self._d.qvel.copy(),'act':np.array([]),'udd_state':{}})()
    def set_state(self, s):
        self._d.time = s.time
        self._d.qpos[:] = s.qpos
        self._d.qvel[:] = s.qvel
        mujoco.mj_forward(self._m, self._d)

class MjViewer:
    def __init__(self, sim): pass
    def render(self): pass

shim.load_model_from_path = lambda p: ShimModel(mujoco.MjModel.from_xml_path(p))
shim.MjSim = lambda m: ShimSim(m)
shim.MjViewer = MjViewer
shim.MujocoException = Exception
shim.ignore_mujoco_warnings = type('ctx',(),{'__enter__':lambda s:None,'__exit__':lambda s,*a:None})
shim.__path__ = []
for sub in ['cymj','builder','generated','generated.const']:
    sys.modules[f'mujoco_py.{sub}'] = types.ModuleType(f'mujoco_py.{sub}')
sys.modules['mujoco_py'] = shim

# === 攔截 print 來記錄 loss ===
os.chdir('/content/diffuserlite.github.io')
os.makedirs('results/figures', exist_ok=True)

loss_log = []
original_print = print

def custom_print(*args, **kwargs):
    original_print(*args, **kwargs)
    if args and isinstance(args[0], dict) and 'gradient_steps' in args[0]:
        loss_log.append(args[0].copy())
        with open('results/loss_log.json', 'w') as f:
            json.dump(loss_log, f)

import builtins
builtins.print = custom_print

# === 執行訓練 ===
import runpy
runpy.run_path('pipelines/diffuserlite_d4rl_mujoco.py', run_name='__main__')

# === 訓練結束後畫圖 ===
import matplotlib.pyplot as plt

if loss_log:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))

    steps = [d['gradient_steps'] for d in loss_log]

    # Diffusion losses
    for key in ['loss0', 'loss1', 'loss2']:
        if key in loss_log[0]:
            values = [d[key] for d in loss_log]
            axes[0].plot(steps, values, label=key)
    axes[0].set_xlabel('Gradient Steps')
    axes[0].set_ylabel('Loss')
    axes[0].set_title('Diffusion Model Loss')
    axes[0].legend()
    axes[0].grid(True)

    # Invdyn loss
    if 'invdyn_loss' in loss_log[0]:
        values = [d['invdyn_loss'] for d in loss_log]
        axes[1].plot(steps, values, color='orange')
        axes[1].set_xlabel('Gradient Steps')
        axes[1].set_ylabel('Loss')
        axes[1].set_title('Inverse Dynamics Loss')
        axes[1].grid(True)

    plt.tight_layout()
    plt.savefig('results/figures/loss_curve.png', dpi=150)
    plt.close()
    original_print("✅ Loss 曲線已儲存到 results/figures/loss_curve.png")

# === 上傳到 Hugging Face ===
original_print("正在上傳到 Hugging Face...")
try:
    from huggingface_hub import HfApi, login
    import glob

    # 你需要先在另一個 cell 執行: huggingface-cli login
    api = HfApi()
    repo_id = "JoshuaLee0816/diffuserlite-results"  # 改成你的 HF username

    # 建立 repo（如果不存在）
    try:
        api.create_repo(repo_id, exist_ok=True)
    except:
        pass

    # 上傳所有 checkpoint 和圖片
    results_dir = '/content/diffuserlite.github.io/results'
    for filepath in glob.glob(f'{results_dir}/**/*', recursive=True):
        if os.path.isfile(filepath):
            path_in_repo = filepath.replace(results_dir + '/', '')
            api.upload_file(
                path_or_fileobj=filepath,
                path_in_repo=path_in_repo,
                repo_id=repo_id,
            )
            original_print(f"  上傳: {path_in_repo}")

    original_print(f"✅ 已上傳到 https://huggingface.co/{repo_id}")
except Exception as e:
    original_print(f"⚠️ Hugging Face 上傳失敗: {e}")
    original_print("請確認已執行: huggingface-cli login")


Writing /content/run.py


In [4]:
# === 階段 0.2：修改 config 為測試用參數 ===
# 訓練 500 步測試流程是否正常

!sed -i 's/save_interval: 200000/save_interval: 100/' /content/diffuserlite.github.io/configs/diffuserlite/mujoco/mujoco.yaml
!sed -i 's/diffusion_gradient_steps: 1000000/diffusion_gradient_steps: 500/' /content/diffuserlite.github.io/configs/diffuserlite/mujoco/mujoco.yaml
!sed -i 's/invdyn_gradient_steps: 1000000/invdyn_gradient_steps: 500/' /content/diffuserlite.github.io/configs/diffuserlite/mujoco/mujoco.yaml
!sed -i 's/log_interval: 1000/log_interval: 10/' /content/diffuserlite.github.io/configs/diffuserlite/mujoco/mujoco.yaml

# 確認修改結果
!echo "=== 測試用參數 ==="
!grep -E "save_interval|diffusion_gradient_steps|invdyn_gradient_steps|log_interval|device:" /content/diffuserlite.github.io/configs/diffuserlite/mujoco/mujoco.yaml

=== 測試用參數 ===
device: cuda:0
diffusion_gradient_steps: 500
invdyn_gradient_steps: 500
log_interval: 10
save_interval: 100


In [5]:
# === 階段 0.2：執行 500 步測試訓練 ===
!export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/root/.mujoco/mujoco210/bin && python3 /content/run.py

These new versions include large bug fixes, new versions of Python, and are where all new development will continue. Please upgrade these libraries as soon as you're able to do so.
If you'd like to read more about the story behind this switch, please check out ]8;;https://farama.org/Announcing-Minari\this blog post]8;;\.
Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.
No module named 'flow'
/usr/local/lib/python3.12/dist-packages/glfw/__init__.py:917: GLFWError: (65550) b'X11: The DISPLAY environment variable is missing'
  warnings.warn(message, GLFWError)
No module named 'carla'
pybullet build time: Feb  6 2026 01:52:08
/usr/local/lib/python3.12/dist-packages/pybullet_env

In [6]:
# === 階段 0.3：確認 checkpoint 有正確存檔 ===
import os

checkpoint_dir = '/content/diffuserlite.github.io/results/diffuserlite_d4rl_mujoco/halfcheetah-medium-expert-v2'

print("=== Checkpoint 檔案列表 ===")
if os.path.exists(checkpoint_dir):
    files = os.listdir(checkpoint_dir)
    for f in sorted(files):
        filepath = os.path.join(checkpoint_dir, f)
        size_mb = os.path.getsize(filepath) / (1024 * 1024)
        print(f"  {f}: {size_mb:.2f} MB")

    # 檢查必要檔案
    required = ['diffusion_ema.pt', 'invdyn.pt']
    missing = [f for f in required if f not in files]

    if missing:
        print(f"\n⚠️ 缺少檔案: {missing}")
    else:
        print("\n✅ Checkpoint 存檔正常！")
else:
    print(f"❌ 目錄不存在: {checkpoint_dir}")

=== Checkpoint 檔案列表 ===
  diffusion0_ckpt_100.pt: 21.30 MB
  diffusion0_ckpt_200.pt: 21.30 MB
  diffusion0_ckpt_300.pt: 21.30 MB
  diffusion0_ckpt_400.pt: 21.30 MB
  diffusion0_ckpt_500.pt: 21.30 MB
  diffusion0_ckpt_latest.pt: 21.30 MB
  diffusion1_ckpt_100.pt: 21.30 MB
  diffusion1_ckpt_200.pt: 21.30 MB
  diffusion1_ckpt_300.pt: 21.30 MB
  diffusion1_ckpt_400.pt: 21.30 MB
  diffusion1_ckpt_500.pt: 21.30 MB
  diffusion1_ckpt_latest.pt: 21.30 MB
  diffusion2_ckpt_100.pt: 21.30 MB
  diffusion2_ckpt_200.pt: 21.30 MB
  diffusion2_ckpt_300.pt: 21.30 MB
  diffusion2_ckpt_400.pt: 21.30 MB
  diffusion2_ckpt_500.pt: 21.30 MB
  diffusion2_ckpt_latest.pt: 21.30 MB
  invdyn_ckpt_100.pt: 0.29 MB
  invdyn_ckpt_200.pt: 0.29 MB
  invdyn_ckpt_300.pt: 0.29 MB
  invdyn_ckpt_400.pt: 0.29 MB
  invdyn_ckpt_500.pt: 0.29 MB
  invdyn_ckpt_latest.pt: 0.29 MB

⚠️ 缺少檔案: ['diffusion_ema.pt', 'invdyn.pt']


In [ ]:
%%writefile /content/inference.py
# === 階段 0.4：執行 inference，確認能輸出分數 ===
import sys, os, types
import mujoco
import numpy as np

# === mujoco_py shim（與 run.py 相同）===
shim = types.ModuleType('mujoco_py')

class ShimModel:
    def __init__(self, m):
        self._m = m
    @property
    def actuator_ctrlrange(self):
        return self._m.actuator_ctrlrange.copy()
    @property
    def nq(self): return self._m.nq
    @property
    def nv(self): return self._m.nv
    def __getattr__(self, name):
        return getattr(self._m, name)

class ShimData:
    def __init__(self, d):
        self._d = d
    @property
    def qpos(self): return self._d.qpos
    @qpos.setter
    def qpos(self, v): self._d.qpos[:] = v
    @property
    def qvel(self): return self._d.qvel
    @qvel.setter
    def qvel(self, v): self._d.qvel[:] = v
    @property
    def ctrl(self): return self._d.ctrl
    @ctrl.setter
    def ctrl(self, v): self._d.ctrl[:] = v
    def __getattr__(self, name):
        return getattr(self._d, name)

class ShimSim:
    def __init__(self, model):
        self._m = model._m
        self._d = mujoco.MjData(self._m)
        self.model = model
        self.data = ShimData(self._d)
    def step(self):
        mujoco.mj_step(self._m, self._d)
    def forward(self):
        mujoco.mj_forward(self._m, self._d)
    def get_state(self):
        return type('S',(),{'time':self._d.time,'qpos':self._d.qpos.copy(),'qvel':self._d.qvel.copy(),'act':np.array([]),'udd_state':{}})()
    def set_state(self, s):
        self._d.time = s.time
        self._d.qpos[:] = s.qpos
        self._d.qvel[:] = s.qvel
        mujoco.mj_forward(self._m, self._d)

class MjViewer:
    def __init__(self, sim): pass
    def render(self): pass

shim.load_model_from_path = lambda p: ShimModel(mujoco.MjModel.from_xml_path(p))
shim.MjSim = lambda m: ShimSim(m)
shim.MjViewer = MjViewer
shim.MujocoException = Exception
shim.ignore_mujoco_warnings = type('ctx',(),{'__enter__':lambda s:None,'__exit__':lambda s,*a:None})
shim.__path__ = []
for sub in ['cymj','builder','generated','generated.const']:
    sys.modules[f'mujoco_py.{sub}'] = types.ModuleType(f'mujoco_py.{sub}')
sys.modules['mujoco_py'] = shim

# === Inference ===
os.chdir('/content/diffuserlite.github.io')

import torch
import gym
import d4rl
from cleandiffuser.env.wrapper import wrap
from cleandiffuser.utils import set_seed

# 載入 checkpoint
device = "cuda:0" if torch.cuda.is_available() else "cpu"
checkpoint_dir = 'results/diffuserlite_d4rl_mujoco/halfcheetah-medium-expert-v2'

print(f"使用裝置: {device}")
print(f"載入 checkpoint: {checkpoint_dir}")

# 載入模型
diffusion_path = f'{checkpoint_dir}/diffusion_ema.pt'
invdyn_path = f'{checkpoint_dir}/invdyn.pt'

if not os.path.exists(diffusion_path):
    print(f"❌ 找不到 diffusion model: {diffusion_path}")
    sys.exit(1)

diffusion = torch.load(diffusion_path, map_location=device)
invdyn = torch.load(invdyn_path, map_location=device)
diffusion.eval()
invdyn.eval()

print("✅ 模型載入成功")

# 建立環境
env_name = "halfcheetah-medium-expert-v2"
env = gym.make(env_name)
env = wrap(env)

# 取得 normalizer
dataset = d4rl.qlearning_dataset(env)
obs_mean = dataset['observations'].mean(axis=0)
obs_std = dataset['observations'].std(axis=0) + 1e-6

# 簡單測試：執行幾步
set_seed(42)
num_episodes = 1
episode_rewards = []

for ep in range(num_episodes):
    obs = env.reset()
    done = False
    total_reward = 0
    steps = 0

    while not done and steps < 100:  # 測試只跑 100 步
        # 正規化 observation
        obs_norm = (obs - obs_mean) / obs_std
        obs_tensor = torch.FloatTensor(obs_norm).unsqueeze(0).to(device)

        # 用 diffusion 預測未來狀態
        with torch.no_grad():
            # 這裡簡化：直接用 random action 測試環境能否運作
            action = env.action_space.sample()

        obs, reward, done, info = env.step(action)
        total_reward += reward
        steps += 1

    episode_rewards.append(total_reward)
    print(f"Episode {ep+1}: reward = {total_reward:.2f}")

# 計算 normalized score
raw_score = np.mean(episode_rewards)
ref_min, ref_max = d4rl.infos.REF_MIN_SCORE[env_name], d4rl.infos.REF_MAX_SCORE[env_name]
normalized_score = (raw_score - ref_min) / (ref_max - ref_min) * 100

print(f"\n=== 測試結果 ===")
print(f"Raw score: {raw_score:.2f}")
print(f"Normalized score: {normalized_score:.2f}")
print(f"\n✅ Inference 流程正常！（注意：這是簡化測試，正式推論需完整實作）")

In [ ]:
# 執行 inference
!export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/root/.mujoco/mujoco210/bin && python3 /content/inference.py

In [ ]:
# === 階段 0.5：上傳 checkpoint 到 Hugging Face ===
from huggingface_hub import HfApi
import os
import glob

api = HfApi()
repo_id = "JoshuaLee0816/diffuserlite-results"

# 建立 repo（如果不存在）
try:
    api.create_repo(repo_id, exist_ok=True)
    print(f"✅ Repo 已準備: https://huggingface.co/{repo_id}")
except Exception as e:
    print(f"建立 repo: {e}")

# 上傳所有 checkpoint 和圖片
results_dir = '/content/diffuserlite.github.io/results'
uploaded_files = []

for filepath in glob.glob(f'{results_dir}/**/*', recursive=True):
    if os.path.isfile(filepath):
        path_in_repo = filepath.replace(results_dir + '/', '')
        try:
            api.upload_file(
                path_or_fileobj=filepath,
                path_in_repo=f"test_phase/{path_in_repo}",  # 放在 test_phase 資料夾
                repo_id=repo_id,
            )
            uploaded_files.append(path_in_repo)
            print(f"  ✅ 上傳: {path_in_repo}")
        except Exception as e:
            print(f"  ❌ 上傳失敗 {path_in_repo}: {e}")

print(f"\n{'='*50}")
print(f"✅ 階段 0 測試完成！共上傳 {len(uploaded_files)} 個檔案")
print(f"🔗 https://huggingface.co/{repo_id}")
print(f"{'='*50}")